<a href="https://colab.research.google.com/github/IshanKashyap01/ML-training/blob/main/Day__10__Kaggle_fake_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd

In [10]:
df=pd.read_csv('/content/train.csv')

In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [12]:
###Drop Nan Values
df=df.dropna()

df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [13]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [14]:
## Get the Dependent features
y=df['label']

In [15]:
X.shape

(18285, 4)

In [16]:
y.shape

(18285,)

In [17]:
import tensorflow as tf

In [18]:
tf.__version__

'2.8.2'

In [19]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [20]:
### Vocabulary size
voc_size=5000

In [21]:
messages=X.copy()

In [22]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [23]:
messages.reset_index(inplace=True)

In [24]:
import nltk
import re
from nltk.corpus import stopwords

In [25]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [26]:
len(messages)

18285

In [27]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

    
    

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [28]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

**Embedding Representation**

In [29]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[77, 1166, 1590, 3996, 3936, 1815, 2986, 3680, 1897, 3369],
 [2632, 3585, 3950, 1126, 511, 4700, 2679],
 [500, 576, 3335, 3143],
 [1337, 2686, 667, 2264, 192, 3643],
 [4833, 511, 2504, 1669, 2624, 3059, 511, 842, 4340, 1294],
 [4498,
  2806,
  581,
  883,
  621,
  3562,
  1736,
  304,
  2555,
  3035,
  695,
  1109,
  4174,
  2658,
  2679],
 [1780, 4929, 958, 1282, 2544, 2509, 1394, 4837, 1342, 4002, 4976],
 [4998, 2281, 4723, 3946, 419, 3047, 3562, 3210, 1342, 4002, 4976],
 [3198, 1530, 3678, 4711, 2214, 1562, 3316, 2099, 3562, 4222],
 [2666, 486, 4912, 1610, 4366, 164, 3875, 4041],
 [4937, 2070, 255, 225, 3982, 1930, 1926, 2766, 2732, 456, 4940],
 [2264, 3998, 3936, 1562, 3562, 419],
 [212, 3866, 3237, 4365, 470, 966, 2320, 1577, 1335],
 [515, 2142, 1364, 566, 4054, 960, 1499, 1342, 4002, 4976],
 [2413, 412, 2558, 2244, 2432, 1342, 4002, 4976],
 [3751, 912, 4955, 1787, 657, 3930, 343, 3160, 996, 3883],
 [750, 490, 3585],
 [2000, 4131, 1119, 4191, 3562, 938, 2876, 2679],
 [880, 1566, 

In [30]:

sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3680 1897 3369]
 [   0    0    0 ...  511 4700 2679]
 [   0    0    0 ...  576 3335 3143]
 ...
 [   0    0    0 ... 1342 4002 4976]
 [   0    0    0 ... 3918 4961 3060]
 [   0    0    0 ... 4837  161 3845]]


In [31]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   77,
       1166, 1590, 3996, 3936, 1815, 2986, 3680, 1897, 3369], dtype=int32)

In [32]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
len(embedded_docs),y.shape

(18285, (18285,))

In [34]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [35]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [37]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 10ms/step - loss: 0.3281 - accuracy: 0.8441 - val_loss: 0.1932 - val_accuracy: 0.9173
Epoch 2/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1411 - accuracy: 0.9449 - val_loss: 0.2077 - val_accuracy: 0.9205
Epoch 3/10
192/192 [==============================] - 1s 7ms/step - loss: 0.1031 - accuracy: 0.9619 - val_loss: 0.2252 - val_accuracy: 0.9145
Epoch 4/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0792 - accuracy: 0.9723 - val_loss: 0.2554 - val_accuracy: 0.9135
Epoch 5/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0587 - accuracy: 0.9793 - val_loss: 0.2656 - val_accuracy: 0.9105
Epoch 6/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0479 - accuracy: 0.9832 - val_loss: 0.3032 - val_accuracy: 0.9123
Epoch 7/10
192/192 [==============================] - 1s 7ms/step - loss: 0.0349 - accuracy: 0.9887 - val_loss: 0.3911 - val_accuracy: 0.9087
Epoch

In [38]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [39]:
predict_x=model.predict(X_test) 
y_pred=np.argmax(predict_x,axis=1)


In [40]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,y_pred)


array([[3419,    0],
       [2616,    0]])

In [41]:

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917